In [1]:
from PIL import Image
import os
import shutil
import numpy as np
# import cv2
import re
import pickle
import random
import time
# from tqdm import tqdm
import pandas as pd 
from glob import glob
from xml.etree.ElementTree import parse
# from pascal_voc_writer import Writer
import matplotlib.pyplot as plt

import os
import numpy as np 
import pandas as pd 
from datetime import datetime
import time
import random
from tqdm.auto import tqdm

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler

#sklearn
from sklearn.model_selection import StratifiedKFold

#CV
import cv2

################# DETR FUCNTIONS FOR LOSS######################## 
import sys
sys.path.extend(['./detr/'])

from models.matcher import HungarianMatcher
from models.detr import SetCriterion
################################################################

#Albumenatations
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2

#Glob
from glob import glob

import sys
sys.path.extend(['./util/'])
from data_parser import *
from image_prepro import *
# from torchvision import transforms

from visualize import *

In [2]:
df = pd.read_csv('df.csv')
df.head()

,file_name,width,height,x_min,y_min,box_width,box_height,org_label,label,label_id,path
0,20200623_100015.jpg,4032,1960,1051,2061,124,117,Suamous cell carcinoma,Carcinoma,1,/home/beomgon/Object_Detection/Dataset/SS/06/2...
1,20200420_100401.jpg,4032,1960,870,1945,71,79,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...
2,20200420_100401.jpg,4032,1960,947,2400,73,84,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...
3,20200420_100401.jpg,4032,1960,845,2209,67,98,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...
4,20200420_100401.jpg,4032,1960,811,2056,55,88,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...


In [3]:
df[df['path'] == '/home/beomgon/Object_Detection/Dataset/SS/04/20200513_104835.png']

,file_name,width,height,x_min,y_min,box_width,box_height,org_label,label,label_id,path
14976,20200513_104835.png,1632,1560,34,1100,93,109,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...
14977,20200513_104835.png,1632,1560,68,1255,88,86,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...
14978,20200513_104835.png,1632,1560,142,1523,113,99,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...
14979,20200513_104835.png,1632,1560,196,1368,97,76,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...
14980,20200513_104835.png,1632,1560,323,1406,122,56,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...
14981,20200513_104835.png,1632,1560,448,1474,109,95,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...
14982,20200513_104835.png,1632,1560,407,1279,107,116,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...
14983,20200513_104835.png,1632,1560,450,1105,79,117,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...
14984,20200513_104835.png,1632,1560,315,1110,81,139,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...
14985,20200513_104835.png,1632,1560,206,994,84,81,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...


In [95]:
# # df.to_csv('df1.csv')
# df = pd.read_csv('df1.csv')
# df.head()
# df.rename(columns = {'Unnamed: 0' : 'index'}, inplace = True)
# df.head()


In [115]:
transforms = A.Compose([
#     A.RandomCrop(width=450, height=450),
#     A.HorizontalFlip(p=1),
    A.CenterCrop(1560,1560, True,1),
#     A.RandomCrop(height=800, width=800, p=0.5),
#     A.Resize(height=512, width=512, p=1),
    A.Resize(height=256, width=256, p=1),
    
    A.OneOf([
    A.HorizontalFlip(p=1),
    A.VerticalFlip(p=1),
#     A.RandomRotate90(p=1),        
    ], p=1),
# #     A.GaussNoise(p=0.5),
#     A.OneOf([
#     A.OpticalDistortion(p=0.7),
#     A.GaussNoise(p=0.7)        
#     ], p=1),
#     A.RandomBrightnessContrast(p=0.2),
#     A.Normalize(max_pixel_value=1),
#     ToTensorV2(p=1.0),
#     A.pytorch.ToTensor(),
# ], p=1.0, bbox_params=A.BboxParams(format='pascal_voc', min_area=0, min_visibility=0.5))
], p=1.0, bbox_params=A.BboxParams(format='coco', min_area=0, min_visibility=0.5, label_fields=['labels'])) 

In [116]:
# transformed = transforms(image=image, bboxes=new_bboxes, labels=labels)
# transformed_image = transformed['image']
# transformed_bboxes = transformed['bboxes']
# transformed_class_labels = transformed['labels']

In [117]:
test_df.head()

,file_name,width,height,x_min,y_min,box_width,box_height,org_label,label,label_id,path,new_box
4000,20200623_130834.jpg,4032,1960,529,1578,77,67,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...,"[(0.2108974358974359, 0.21923076923076923, 0.0..."
4001,20200623_130834.jpg,4032,1960,501,1760,65,78,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...,"[(0.19294871794871796, 0.33589743589743587, 0...."
4002,20200623_130834.jpg,4032,1960,582,1747,78,75,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...,"[(0.24487179487179486, 0.32756410256410257, 0...."
4003,20200623_130834.jpg,4032,1960,922,2366,83,70,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...,"[(0.46282051282051284, 0.7243589743589743, 0.0..."
4004,20200623_130834.jpg,4032,1960,931,2261,84,89,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...,"[(0.4685897435897436, 0.657051282051282, 0.053..."


In [118]:
new_box = []
test_df = df[4000:6000]
# for i in range (len(df)) :
for i in range(len(test_df)) :
    i += 4000
    path = test_df.loc[i, 'path']
    x = test_df.loc[i, 'x_min']
    y = test_df.loc[i, 'y_min']
    w = test_df.loc[i, 'box_width']
    h = test_df.loc[i, 'box_height']
    boxes = [[x, y, w, h]]
#     print(boxes)
    labels = ['normal']
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.flip(image, 1)
#     print(image.shape)
    transformed = transforms(image=image, bboxes=boxes, labels=labels)
    new_box.append(transformed['bboxes'])
    

In [119]:
test_df['new_box'] = new_box

<ipython-input-119-8c40891c6d23>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['new_box'] = new_box


In [120]:
test_df.head()

,file_name,width,height,x_min,y_min,box_width,box_height,org_label,label,label_id,path,new_box
4000,20200623_130834.jpg,4032,1960,529,1578,77,67,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...,"[(189.37435897435898, 56.12307692307692, 12.63..."
4001,20200623_130834.jpg,4032,1960,501,1760,65,78,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...,"[(195.93846153846152, 85.98974358974358, 10.66..."
4002,20200623_130834.jpg,4032,1960,582,1747,78,75,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...,"[(180.5128205128205, 83.85641025641026, 12.800..."
4003,20200623_130834.jpg,4032,1960,922,2366,83,70,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...,"[(118.48205128205129, 59.076923076923066, 13.6..."
4004,20200623_130834.jpg,4032,1960,931,2261,84,89,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/06/2...,"[(122.25641025641025, 168.2051282051282, 13.78..."


In [121]:
# test_df.new_box[0]

In [122]:
# #         _,h,w = image.shape
#         h = w = 256
#         boxes = A.augmentations.bbox_utils.normalize_bboxes(sample['bboxes'],rows=h,cols=w)
#         ## detr uses center_x,center_y,width,height !!
#         if len(boxes)>0:
#             boxes = np.array(boxes)
#             boxes[:,2:] /= 2
#             boxes[:,:2] += boxes[:,2:]

In [123]:
h = w = 256
test_df['new_box'] = test_df['new_box'].apply(lambda x : A.augmentations.bbox_utils.normalize_bboxes(x,rows=h,cols=w))

<ipython-input-123-f6d480f96bb8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['new_box'] = test_df['new_box'].apply(lambda x : A.augmentations.bbox_utils.normalize_bboxes(x,rows=h,cols=w))


In [124]:
test_df['new_box']

4000    [(0.7397435897435898, 0.21923076923076923, 0.0...
4001    [(0.7653846153846153, 0.33589743589743587, 0.0...
4002    [(0.7051282051282051, 0.32756410256410257, 0.0...
4003    [(0.46282051282051284, 0.23076923076923073, 0....
4004    [(0.47756410256410253, 0.657051282051282, 0.05...
                              ...                        
5995    [(0.5108974358974359, 0.41217948717948716, 0.0...
5996    [(0.7506410256410256, 0.4596153846153846, 0.07...
5997    [(0.5935897435897436, 0.48205128205128206, 0.0...
5998    [(0.4833333333333333, 0.3371794871794872, 0.08...
5999    [(0.366025641025641, 0.4955128205128204, 0.110...
Name: new_box, Length: 2000, dtype: object

In [107]:
# def box_chanage(boxes) :
# #     print(boxes)
#     if len(boxes) > 0 :
#         boxes = np.array(boxes)
#         boxes[:,2:] /= 2
#         boxes[:,:2] += boxes[:,2:]
#         return boxes
#     else :
#         return []
# test_df['new_box'] = test_df['new_box'].apply(lambda x : box_chanage(x))

<ipython-input-107-6ab27a7a1e76>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['new_box'] = test_df['new_box'].apply(lambda x : box_chanage(x))


In [109]:
test_df['new_box']

2000    [[0.5743589743589743, 0.635576923076923, 0.020...
2001    [[0.38429487179487176, 0.5794871794871794, 0.0...
2002    [[0.4128205128205128, 0.5253205128205128, 0.02...
2003    [[0.5003205128205128, 0.514102564102564, 0.027...
2004    [[0.5599358974358974, 0.551923076923077, 0.022...
                              ...                        
3995    [[0.48141025641025637, 0.6919871794871795, 0.0...
3996    [[0.6900641025641026, 0.6333333333333333, 0.02...
3997    [[0.46442307692307694, 0.7541666666666667, 0.0...
3998    [[0.3192307692307692, 0.6605769230769231, 0.03...
3999    [[0.5884615384615384, 0.5875, 0.01858974358974...
Name: new_box, Length: 2000, dtype: object

In [92]:
820/4032

0.20337301587301587

In [91]:
# for i in range(len(test_df)) :
# #     path = df.loc[i, 'path']  
#     if len(test_df.loc[i, 'new_box']) > 0 :
#         box = test_df.loc[i, 'new_box'][0]
#         for b in box :
#             if b >= 1 :
#                 print(i)

In [9]:
# df_data = df.groupby('file_name')
# images = list(set(df.file_name))
df_data = df.groupby('index')
# df_data = df.groupby('index')
images = list(set(df.index))
len(images)
# len(df_data)
# len(df_data.groups)
# df_data.get_group('20200420_100401.jpg')

101932

In [10]:
def get_data(img_id):
    if img_id not in df_data.groups:
        return dict(image_id=img_id, source='', boxes=list())
    
    data  = df_data.get_group(img_id)
    labels = data.label.values
#     source = data.source.values
#     source = np.unique(data.source.values)
#     assert len(source)==1, 'corrupted data: %s image_id has many sources: %s' %(img_id,source)
#     source=source[0]
    boxes = data[['x_min','y_min','box_width','box_height']].values
#     boxes = list(data.bbox)
    label_id = data.label_id.values
    width = data.width.values
    height = data.height.values
    return dict(image_id = img_id, labels=labels, 
                label_id=label_id, boxes = boxes,
               width = width, height=height)

image_list = [get_data(img_id) for img_id in images]

In [19]:
# image_list[0]

{'image_id': 0,
 'labels': array(['Carcinoma'], dtype=object),
 'label_id': array([1]),
 'boxes': array([[1051, 2061,  124,  117]]),
 'width': array([4032]),
 'height': array([1960])}

In [11]:
# transforms = A.Compose([
# #     A.RandomCrop(width=450, height=450),
# #     A.HorizontalFlip(p=1),
#     A.CenterCrop(1560,1560, True,1),
# #     A.RandomCrop(height=800, width=800, p=0.5),
# #     A.Resize(height=512, width=512, p=1),
#     A.Resize(height=256, width=256, p=1),
    
#     A.OneOf([
#     A.HorizontalFlip(p=0.9),
#     A.VerticalFlip(p=0.9),
#     A.RandomRotate90(p=0.9),        
#     ], p=1),
# #     A.GaussNoise(p=0.5),
# #     A.OneOf([
# #     A.OpticalDistortion(p=0.7),
# #     A.GaussNoise(p=0.7)        
# #     ], p=1),
# #     A.RandomBrightnessContrast(p=0.2),
# #     A.Normalize(max_pixel_value=1),
# #     ToTensorV2(p=1.0),
#     A.pytorch.ToTensor(),
# # ], p=1.0, bbox_params=A.BboxParams(format='pascal_voc', min_area=0, min_visibility=0.5))
# ], p=1.0, bbox_params=A.BboxParams(format='coco', min_area=0, min_visibility=0.5, label_fields=['labels'])) 

In [20]:
# DIR_TRAIN = '../input/global-wheat-detection/train'
class WheatDataset(Dataset):
    def __init__(self,image_list,transforms=None):
        self.images = image_list
        self.transforms = transforms
        self.img_ids = {x['image_id']:i for i,x in enumerate(image_list)}
        
    def get_indices(self,img_ids):
        return [self.img_ids[x] for x in img_ids]
        
    def __len__(self) -> int:
        return len(self.images)
    
    def __getitem__(self,index):
        record = self.images[index]
        image_id = record['image_id']

        image = cv2.imread(image_id, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image = cv2.flip(image, 1)
        image /= 255.0
        
        # DETR takes in data in coco format 
        boxes = record['boxes'] 
        
#         labels =  np.zeros(len(boxes), dtype=np.int32)
        labels = record['label_id']
        width = record['width']
        height = record['height']

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': boxes,
                'labels': labels
            }
            sample = self.transforms(**sample)
            image  = sample['image']
            boxes  = sample['bboxes']
            labels = sample['labels']

        h,w, _ = image.shape
        boxes = A.augmentations.bbox_utils.normalize_bboxes(sample['bboxes'],rows=h,cols=w)
        ## detr uses center_x,center_y,width,height !!
        if len(boxes)>0:
            boxes = np.array(boxes)
            boxes[:,2:] /= 2
            boxes[:,:2] += boxes[:,2:]
        else:
            boxes = np.zeros((0,4))
    
        target = {}
        target['boxes'] = torch.as_tensor(boxes,dtype=torch.float32)
        target['labels'] = torch.as_tensor(labels, dtype=torch.int64)
        target['image_id'] = torch.tensor([index])
        
#         target['width'] = torch.tensor([width])
#         target['height'] = torch.tensor([height])
        
        return image, target, image_id, width, height

In [21]:
train_ds = WheatDataset(image_list,transforms)
# valid_ds = WheatDataset(image_list,get_valid_transforms())

In [26]:
class Reader(object):
    def __init__(self, g):
        self.g = g
    def read(self, n=0):
        try:
            return next(self.g)
        except StopIteration:
            return ''

In [28]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [29]:
train_data_loader = DataLoader(
#     torch.utils.data.Subset(train_ds,train_indexes),
    train_ds,
    batch_size=1,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
#     collate_fn=None
)

In [10]:
class DETRModel(nn.Module):
    def __init__(self,num_classes,num_queries):
        super(DETRModel,self).__init__()
        self.num_classes = num_classes
        self.num_queries = num_queries
        
        self.model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
        self.in_features = self.model.class_embed.in_features
        
        self.model.class_embed = nn.Linear(in_features=self.in_features,out_features=self.num_classes+1)
        self.model.num_queries = self.num_queries
        
    def forward(self,images):
        return self.model(images)

In [11]:
n_folds = 5
seed = 42
null_class_coef = 0.5
# num_classes = 1
num_queries = 100
# BATCH_SIZE = 2
LR = 5e-5
lr_dict = {'backbone':0.1,'transformer':1,'embed':1,'final': 5}
EPOCHS = 2
max_norm = 0
model_name = 'detr_resnet50'

In [12]:
'''
code taken from github repo detr , 'code present in engine.py'
'''

# matcher = HungarianMatcher(cost_giou=2,cost_class=1,cost_bbox=5)
matcher = HungarianMatcher()

weight_dict = {'loss_ce': 1, 'loss_bbox': 5 , 'loss_giou': 2}

losses = ['labels', 'boxes', 'cardinality']

In [13]:
num_classes = 5
criterion = SetCriterion(num_classes, 
                         matcher, weight_dict, 
                         eos_coef = null_class_coef, 
                         losses=losses)

In [19]:
device = torch.device('cuda')
criterion = criterion.to(device)
model = DETRModel(num_classes=5, num_queries=num_queries)
model = model.to(device)
model.train()
criterion.train()
tk0 = tqdm(train_data_loader, total=len(train_data_loader),leave=False)
log = None
device='cuda'
for step, (images, targets, image_ids, width, height) in enumerate(tk0):
#     print(image_ids, width[0][0], height[0][0])
    images = list(image.to(device) for image in images)
#     print(images[0])
#     print(type(images))
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
    
    output = model(images)

    loss_dict = criterion(output, targets)
# #     dfsdfs
#     if log is None:
#         log = {k:AverageMeter() for k in loss_dict}
#         log['total_loss'] = AverageMeter()
#         log['avg_prec'] = AverageMeter()

#     weight_dict = criterion.weight_dict

#     total_loss = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)

#     optimizer.zero_grad()

#     total_loss.backward()

# #     if max_norm > 0:
# #         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

#     optimizer.step()    
    
    

Using cache found in /home/beomgon/.cache/torch/hub/facebookresearch_detr_master


ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/beomgon/anaconda3/envs/detr/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 202, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/beomgon/anaconda3/envs/detr/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/beomgon/anaconda3/envs/detr/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "<ipython-input-6-1596775de7e6>", line 37, in __getitem__
    sample = self.transforms(**sample)
  File "/home/beomgon/anaconda3/envs/detr/lib/python3.8/site-packages/albumentations/core/composition.py", line 180, in __call__
    p.preprocess(data)
  File "/home/beomgon/anaconda3/envs/detr/lib/python3.8/site-packages/albumentations/core/uti